# 데이터 읽어오기

In [3]:
import pandas as pd
import numpy as np
movies = pd.read_csv("./dataset/tmdb_5000_movies.csv")         # tmdb_5000_movies.csv dataframe으로 읽어오기
movies = movies[["id","genres","vote_average","vote_count","popularity","original_title", "keywords","overview"]]
movies.drop(["genres","vote_average","vote_count","popularity", "keywords","overview"],axis=1,inplace=True)

print(movies.shape)
movies.head()

(4803, 2)


,id,original_title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [4]:
# 컬럼명 변경
movies.columns = ['tmdbId', 'original_title']
movies.head()

,tmdbId,original_title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [5]:
# links.csv dataframe으로 읽어오기
links = pd.read_csv("./dataset/links_small.csv")
links = links[["movieId","imdbId","tmdbId"]]
links.drop(['imdbId'],axis=1,inplace=True)

print(links.shape)
links.head()

(9125, 2)


,movieId,tmdbId
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0


In [6]:
# ratings.csv dataframe으로 읽어오기
ratings = pd.read_csv("./dataset/ratings_small.csv")   
ratings = ratings[["userId","movieId","rating","timestamp"]]
ratings.drop(["timestamp"],axis=1,inplace=True)

print(ratings.shape)
ratings.head()

(100004, 3)


,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


## 결측값 확인


In [7]:
# null 값 개수 확인
movies.isnull().sum()

tmdbId            0
original_title    0
dtype: int64

In [8]:
# null 값 개수 확인
links.isnull().sum()

movieId     0
tmdbId     13
dtype: int64

In [9]:
# 결측값 있는 로우 제거
links = links.dropna(axis=0)

# 타입변환
links["tmdbId"] = links["tmdbId"].astype("int")

links.head()

,movieId,tmdbId
0,1,862
1,2,8844
2,3,15602
3,4,31357
4,5,11862


In [10]:
# null 값 개수 확인
ratings.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

## 데이터 프레임끼리 inner join

In [11]:
join_movie = pd.merge(ratings, links, left_on='movieId', right_on='movieId', how='inner')
print(join_movie)

       userId  movieId  rating  tmdbId
0           1       31     2.5    9909
1           7       31     3.0    9909
2          31       31     4.0    9909
3          32       31     4.0    9909
4          36       31     3.0    9909
...       ...      ...     ...     ...
99928     664    64997     2.5   34812
99929     664    72380     3.5   22825
99930     665      129     3.0  110972
99931     665     4736     1.0   26602
99932     668     6425     1.0   36807

[99933 rows x 4 columns]


In [12]:
join_movie = pd.merge(join_movie, movies, left_on='tmdbId', right_on='tmdbId', how='inner')
print(join_movie)

       userId  movieId  rating  tmdbId  original_title
0           1     1061     3.0     819        Sleepers
1          19     1061     3.0     819        Sleepers
2          23     1061     3.5     819        Sleepers
3          30     1061     3.0     819        Sleepers
4          70     1061     5.0     819        Sleepers
...       ...      ...     ...     ...             ...
66942     663   134528     3.5  222936           Aloha
66943     663   137595     3.0  264999  Magic Mike XXL
66944     664    60832     3.0   12192       Pathology
66945     664    72380     3.5   22825         The Box
66946     665     4736     1.0   26602    Summer Catch

[66947 rows x 5 columns]


In [13]:
join_movie["original_title"].value_counts()

Forrest Gump                341
Pulp Fiction                324
The Shawshank Redemption    311
The Silence of the Lambs    304
Star Wars                   291
                           ... 
Isn't She Great               1
Monsters                      1
FearDotCom                    1
Cheap Thrills                 1
Mr. Peabody & Sherman         1
Name: original_title, Length: 3393, dtype: int64

In [14]:
# # movies에 있던 모든 영화가 join이 잘 됐는지 확인 -> 여긴 발표할 때는 빼도 될 듯..
# after = join_movie["tmdbId"].unique()
# print(len(after))
# print(movies.shape)

## latent 알고리즘

### 피벗테이블 생성

In [15]:
user_movie_rating = join_movie.pivot_table('rating', index = 'userId', columns='tmdbId').fillna(0)
# user_movie_rating = join_movie.pivot_table('rating', index = 'userId', columns='tmdbId')
print(user_movie_rating.shape)
user_movie_rating.head()

(671, 3394)


tmdbId,5,11,12,13,14,16,18,19,20,22,24,25,28,33,35,38,55,58,59,62,65,66,68,69,70,71,73,74,75,76,77,78,79,80,83,85,87,89,90,95,...,291270,291805,293660,293863,294086,294254,295964,296096,296098,296099,297596,297761,298312,301748,302699,303858,307081,308531,308639,310131,312221,313922,314365,316727,318846,321258,321697,321741,324668,325133,325173,328111,328425,329833,332411,332567,333371,334074,342521,347969
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,3.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 특이값 분해 (Singular Value Decomposion, SVD)
행렬을 여러 행렬의 곱으로 분해해준다. </br>
[참고 사이트](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/04/06/pcasvdlsa/)

scipy의 Truncated SVD 이용 </br>

반환값: U 행렬, Sigma 행렬, V 전치 행렬(Vt)



In [16]:
from scipy.sparse.linalg import svds
import numpy as np

# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(user_movie_rating, k = 100)


print(U.shape)
print(sigma.shape)
print(Vt.shape)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1883: RuntimeWarning: invalid value encountered in greater
  above_cutoff = (eigvals > cutoff)


(671, 100)
(100,)
(100, 3394)



현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태->0이 포함된 대칭행렬로 변환할 때는 numpy의 diag를 이용

In [17]:
sigma = np.diag(sigma)

tmp = pd.DataFrame(data=sigma, index=None, columns=None, dtype=None, copy=False)
print(tmp.shape)
tmp.head()

(100, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 원래 행렬로 복구

U, Sigma, Vt의 내적을 수행

In [18]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 

svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 


In [19]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_movie_rating.columns)
print(df_svd_preds.shape)
df_svd_preds.head()

(671, 3394)


tmdbId,5,11,12,13,14,16,18,19,20,22,24,25,28,33,35,38,55,58,59,62,65,66,68,69,70,71,73,74,75,76,77,78,79,80,83,85,87,89,90,95,...,291270,291805,293660,293863,294086,294254,295964,296096,296098,296099,297596,297761,298312,301748,302699,303858,307081,308531,308639,310131,312221,313922,314365,316727,318846,321258,321697,321741,324668,325133,325173,328111,328425,329833,332411,332567,333371,334074,342521,347969
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.metrics import mean_squared_error


# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수
def calculate_rmse(R, P, Q, non_zeros):
    error = 0

    full_pred_matrix = np.dot(P, Q.T)

    # 여기서 non_zeros는 아래 함수에서 확인할 수 있다.
    x_non_zero_ind = [non_zeros[0] for non_zeros in non_zeros]
    y_non_zero_ind = [non_zeros[1] for non_zeros in non_zeros]

    # 원 행렬 R에서 0이 아닌 값들만 추출한다.
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    # 예측 행렬에서 원 행렬 R에서 0이 아닌 위치의 값들만 추출하여 저장한다.
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse


def matrix_factorization(R, K, steps=100, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape

    np.random.seed(1)
    P = np.random.normal(scale=1.0/K, size=(num_users, K))
    Q = np.random.normal(scale=1.0/K, size=(num_items, K))

    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다.
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users)
                  for j in range(num_items) if R[i, j] > 0 ]

    # SGD 기법으로 P, Q 매트릭스를 업데이트 함
    for step in range(steps):
        for i, j, r in non_zeros:
            # 잔차 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)

            # Regulation을 반영한 SGD 업데이터 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

        rmse = calculate_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print("iter step: {0}, rmse: {1:4f}".format(step, rmse))

    return P, Q

P, Q = matrix_factorization(user_movie_rating.values, K=3)
pred_matrix = np.dot(P, Q.T)
print(pred_matrix)

iter step: 0, rmse: 2.615706
iter step: 10, rmse: 0.856848
iter step: 20, rmse: 0.813864
iter step: 30, rmse: 0.793731
iter step: 40, rmse: 0.780472
iter step: 50, rmse: 0.770871
iter step: 60, rmse: 0.764027
iter step: 70, rmse: 0.759228
iter step: 80, rmse: 0.755759
iter step: 90, rmse: 0.753154
iter step: 100, rmse: 0.751162
iter step: 110, rmse: 0.749618
iter step: 120, rmse: 0.748390
iter step: 130, rmse: 0.747387
iter step: 140, rmse: 0.746549
iter step: 150, rmse: 0.745839
iter step: 160, rmse: 0.745231
iter step: 170, rmse: 0.744706
iter step: 180, rmse: 0.744249
iter step: 190, rmse: 0.743848
[[1.76897411 3.1544787  2.15897124 ... 1.32105094 0.88107398 1.21498791]
 [3.60024157 3.92663784 4.03945049 ... 2.8695976  0.37581765 2.19400046]
 [3.41100741 4.17208103 4.09017623 ... 2.93801356 0.34225858 2.20535621]
 ...
 [3.12617473 4.29722409 3.90657301 ... 2.77318986 0.45949023 2.10885494]
 [2.82209349 4.28611049 3.85550551 ... 2.8175759  0.27704741 2.05306369]
 [3.62765276 4.222228

In [21]:
df_svd_preds = pd.DataFrame(pred_matrix, columns = user_movie_rating.columns)
print(df_svd_preds.shape)
df_svd_preds.head()

(671, 3394)


tmdbId,5,11,12,13,14,16,18,19,20,22,24,25,28,33,35,38,55,58,59,62,65,66,68,69,70,71,73,74,75,76,77,78,79,80,83,85,87,89,90,95,...,291270,291805,293660,293863,294086,294254,295964,296096,296098,296099,297596,297761,298312,301748,302699,303858,307081,308531,308639,310131,312221,313922,314365,316727,318846,321258,321697,321741,324668,325133,325173,328111,328425,329833,332411,332567,333371,334074,342521,347969
0,1.768974,3.154479,2.158971,2.390446,2.452858,1.220180,2.811815,3.242590,2.800770,2.316559,2.725164,1.774845,3.034698,2.968916,1.497164,3.097589,2.731976,1.242667,1.228197,3.820898,1.818501,1.358762,1.670260,1.936077,1.867997,1.991612,2.678467,1.049541,2.987245,3.399195,3.065424,3.342599,2.381963,3.404708,1.309195,3.223765,2.833766,2.504346,2.916708,1.650190,...,1.683010,1.031566,1.637042,2.604335,2.695632,1.684008,1.693919,1.133124,2.136855,0.409245,1.045776,1.992527,2.360226,0.866715,2.164623,1.306232,2.665797,1.701795,0.961844,3.117406,2.729268,2.718359,2.937714,1.703147,2.701351,2.774889,1.376926,0.828542,2.258801,2.912242,1.790643,0.984677,1.978576,0.875069,1.756602,2.999833,1.914619,1.321051,0.881074,1.214988
1,3.600242,3.926638,4.039450,3.970509,3.784446,3.643142,3.425553,3.214228,4.103898,3.931669,3.556206,3.222999,3.233717,3.146914,3.616318,3.549961,3.995832,3.806345,3.922788,2.982084,3.624550,3.170190,3.970552,3.978957,3.715259,3.531379,3.963761,3.204160,2.679193,3.853189,3.805555,3.399272,3.694943,3.749523,3.308499,3.894511,3.561555,3.848064,3.489152,3.180257,...,1.968688,3.046562,3.626525,2.094075,4.183902,3.213652,1.236575,3.771489,3.910014,3.035078,1.482601,2.235245,3.215231,1.449144,1.113651,0.462360,3.348971,2.309036,3.032541,2.866291,3.277632,3.568733,4.122927,1.075634,3.590930,2.136020,2.961666,1.458851,3.412701,3.563442,2.451305,3.684759,3.837566,2.299321,1.603184,3.075885,3.786658,2.869598,0.375818,2.194000
2,3.411007,4.172081,4.090176,4.092620,4.148720,3.966574,3.542530,3.670857,4.030797,3.976766,3.983329,3.658814,3.764427,3.629322,3.761421,3.987226,4.297277,3.695142,4.054221,3.504622,3.685095,3.065930,4.275110,4.048924,3.955102,3.860515,4.212961,3.153133,2.939438,4.001150,4.214020,3.837770,4.069939,3.971696,3.647859,4.118695,3.747349,3.967889,3.646037,2.816071,...,2.114239,3.171076,3.607803,2.154608,4.580087,2.820136,1.578458,3.840881,3.984077,2.840791,1.567536,2.668899,3.300975,1.562317,1.148148,0.457843,3.338447,2.455248,3.376755,3.436839,3.864871,3.701357,4.353623,1.158320,3.756339,2.279436,2.867423,1.562782,3.445058,3.932931,2.835285,3.563347,3.948549,2.412795,1.578946,3.175814,4.028075,2.938014,0.342259,2.205356
3,4.016853,5.032875,4.849932,4.876290,4.971751,4.674654,4.272331,4.486454,4.811415,4.729691,4.807136,4.371075,4.588078,4.423091,4.437991,4.837915,5.154316,4.316285,4.754627,4.341898,4.362686,3.602134,5.057410,4.789604,4.696028,4.605618,5.048063,3.687286,3.607282,4.842286,5.093069,4.682940,4.877649,4.817039,4.316332,4.973883,4.516704,4.740461,4.402737,3.306063,...,2.558507,3.726108,4.252859,2.657765,5.487204,3.309773,1.965527,4.496273,4.729144,3.269355,1.880721,3.248450,3.960869,1.865743,1.469043,0.613740,4.013920,2.951563,3.981968,4.221783,4.689061,4.450185,5.225121,1.454858,4.515822,2.821858,3.376448,1.862673,4.113609,4.755514,3.420428,4.146319,4.681717,2.843276,1.929808,3.861915,4.783178,3.471757,0.449008,2.616676
4,4.309973,4.308067,4.625779,4.481545,4.073296,3.981710,3.831570,3.305372,4.775269,4.498243,3.747361,3.382454,3.277967,3.215845,4.078960,3.719318,4.360411,4.509455,4.458433,2.961924,4.143099,3.753644,4.364974,4.547573,4.108788,3.819528,4.365139,3.762894,2.838692,4.289033,4.042657,3.533098,3.962412,4.109662,3.573639,4.284465,3.937026,4.335718,3.871627,3.956955,...,2.138980,3.442479,4.213144,2.320231,4.508868,4.017715,1.127349,4.332299,4.456574,3.697690,1.637634,2.217654,3.624596,1.582660,1.205232,0.500926,3.845623,2.537162,3.258071,2.813109,3.296200,3.990660,4.557296,1.139183,3.991868,2.299292,3.500582,1.602534,3.901192,3.794088,2.516650,4.383615,4.348402,2.579679,1.839244,3.428